In [13]:
from dotenv import load_dotenv
from polygon import RESTClient
from datetime import datetime,timezone
import os

In [14]:
import pandas as pd

In [15]:
import numpy as np

In [16]:

load_dotenv("../guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [17]:

client = RESTClient(api_key=token)

In [26]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2023-12-30", limit=50000):
    aggs.append(a)

print(len(aggs))



393359


In [27]:
aggs[-1]

Agg(open=192.1505, high=192.1505, low=192.1501, close=192.1501, volume=2147, vwap=192.1512, timestamp=1703897940000, transactions=12, otc=None)

In [28]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



In [29]:
from heston_param import *

In [30]:
aapl_aggs.groupby("day")

In [31]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [32]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day





In [33]:
# import pickle
# from heston_param import *
# hist_voilatilities = pickle.load(open("hist_voilatilities.pkl", "rb"))
# daily_aggs = pickle.load(open("daily_aggs.pkl", "rb"))

In [34]:
print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, daily_aggs[0]["close"].values.shape[0], 300))

[ 1.89209127  0.04877314  0.60208192 -0.75079845  0.03034837]


In [35]:
while True:
	try:
		print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, daily_aggs[0]["close"].values.shape[0], 300))
		break
	except RuntimeError:
		continue

[5.99403759 0.10262145 1.27470711 0.69184232 0.64249154]


In [36]:
def single_day_calibration(args):
		i, day = args
		while True:
			try:
				params = calibrate_daily_parameters(hist_volatilities[i], 0.1, day["close"].values, 0.0237, day["close"].values.shape[0], 300)
				
				return i, params
			except RuntimeError:
				continue

In [44]:
daily_params = np.empty((len(daily_aggs), 5))
for i, day in enumerate((daily_aggs)):
	while True:
		try:
			daily_params[i] = calibrate_daily_parameters(hist_voilatilities[i], 0.1, daily_aggs[i]["close"].values, 0.0237, daily_aggs[i]["close"].values.shape[0], 300)
			if (i + 1) % 10 == 0:
					rate = np.round(100 * (i + 1) /day["close"].values.shape[0], 2)
					print(f"{rate}% completed.")
			
			break
		except RuntimeError:
			continue

daily_params

KeyboardInterrupt: 